# Predictive Analysis

------------

# 4. Utilization Prediction

In this section we predict the absolute Utilization (number of cars per hour) as well as the percentage Utilization (percentage of possible charging minutes occupied)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras_tuner import HyperModel, RandomSearch
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

KeyboardInterrupt: 

In [ ]:
charging_set_complete = pd.read_csv("data/cleaned_charging_set.csv")

In [ ]:
charging_set_complete.info()

In [ ]:
weather_set = pd.read_csv("data/cleaned_weather_set.csv")

In [ ]:
weather_set.info()

In [ ]:
print(weather_set["timestamp"].max(),charging_set_complete["connectionTime"].max())
print(weather_set["timestamp"].min(),charging_set_complete["connectionTime"].min())

As we can see the datasets have different time frames, whcih we have to keep in mind for merging later. Unfortunetly this means some data will be lost. 

In [ ]:
# seperate data into site 1 and site 2
charging_set_site1 = charging_set_complete[charging_set_complete["siteID"] == 1]
charging_set_site2 = charging_set_complete[charging_set_complete["siteID"] == 2]

In [ ]:
charging_set_site1.head()

In [ ]:
df_no_duplicates = charging_set_site1.drop_duplicates()

In [ ]:
# check space amount of each parking site 
amount_spaces1 = charging_set_site1["spaceID"].nunique()
amount_spaces2 = charging_set_site2["spaceID"].nunique()
print(amount_spaces1,amount_spaces2)

Let's create a dataframe that contains an entry for every hour in between the first and last charging event. This will becime usefull for the prediction later, as the task is to specifically predict hourly utiliaztion.

This has to be done for Site 1 and 2. The procedure is the same so it will look redundant

In [ ]:
charging_set_site1["connectionTime"] = pd.to_datetime(charging_set_site1["connectionTime"])
charging_set_site1["disconnectTime"] = pd.to_datetime(charging_set_site1["disconnectTime"])
# sort acsending
charging_set_site1 = charging_set_site1.sort_values(by='connectionTime').reset_index(drop=True)

# set min and max time 
min_time = charging_set_site1['connectionTime'].min().floor('H')
max_time = charging_set_site1['disconnectTime'].max().ceil('H')

# create df for every hour 
hours_df = pd.DataFrame({'hour': pd.date_range(start=min_time, end=max_time, freq='H')})
hours_df.head()

In [ ]:
charging_set_site2["connectionTime"] = pd.to_datetime(charging_set_site2["connectionTime"])
charging_set_site2["disconnectTime"] = pd.to_datetime(charging_set_site2["disconnectTime"])

# sort ascending
charging_set_site2 = charging_set_site2.sort_values(by='connectionTime').reset_index(drop=True)

# set min and max time 
min_time = charging_set_site2['connectionTime'].min().floor('H')
max_time = charging_set_site2['disconnectTime'].max().ceil('H')

# create df for every hour 
hours_df2 = pd.DataFrame({'hour': pd.date_range(start=min_time, end=max_time, freq='H')})
hours_df2.head()


We can see that there are different timeframes of data available for both sites. 

In [ ]:
# check for duplicates to make sure df creation was succesful 
print(hours_df.duplicated().sum(), hours_df2.duplicated().sum())

0 0


In [ ]:
# check for duplicates in charging sets
print(charging_set_site1.duplicated().sum(), charging_set_site2.duplicated().sum())


1412 0


In [ ]:
# drop duplicates we foundin site 1
charging_set_site1 = charging_set_site1.drop_duplicates()
charging_set_site1.duplicated().sum()

0

Now we want to create our target variables that we want to predict later. For this we create 2 values: 
- connectedCars: the number of active charging sessions for each other
- utilization: the actual number of minutes that was charged for each hour 

The utiliaztion column effectivly sums up the active charging minutes for each hour and divides that by the maximum number of minutes that could be charged. In case of sitze 1 for example that would mean 60x52 = 3120 are the max charging miinutes for this site per hour. If we divide the minutes of active sessions by this value, we get the precise percentage of the charging site utilization.

**Calculation for Site 1**

In [2]:
# number of spaces (total available spaces site 1)
total_spaces = 52

# function to calculate active session minutes for each hour
def calculate_utilization(hour, charging_set_site1, total_spaces):
    start_of_hour = hour
    end_of_hour = hour + pd.Timedelta(hours=1)
    
    # filter sessions active during this hour
    active_sessions = charging_set_site1[
        (charging_set_site1['connectionTime'] < end_of_hour) & (charging_set_site1['disconnectTime'] > start_of_hour)
    ]
    
    # calculate active minutes for each session
    active_minutes = 0
    for _, session in active_sessions.iterrows():
        session_start = max(session['connectionTime'], start_of_hour)
        session_end = min(session['disconnectTime'], end_of_hour)
        active_minutes += (session_end - session_start).total_seconds() / 60  # Convert to minutes
    
    # count number of cars
    connected_cars = len(active_sessions)
    # utilization as a percentage
    possible_minutes = total_spaces * 60  # 60 minutes per hour per space
    utilization = active_minutes / possible_minutes if possible_minutes > 0 else 0
    return utilization, connected_cars


# compute utilization column
hours_df['utilization'] = hours_df['hour'].apply(
    lambda h: calculate_utilization(h, charging_set_site1, total_spaces)[0]
)

# compute connectedCars column
hours_df['connectedCars'] = hours_df['hour'].apply(
    lambda h: calculate_utilization(h, charging_set_site1, total_spaces)[1]
)

# display results
hours_df.head()

NameError: name 'hours_df' is not defined

In [3]:
# number of spaces (total available spaces site 1)
total_spaces = 54

# function to calculate active session minutes for each hour
def calculate_utilization(hour, charging_set_site2, total_spaces):
    start_of_hour = hour
    end_of_hour = hour + pd.Timedelta(hours=1)
    
    # filter sessions active during this hour
    active_sessions = charging_set_site2[
        (charging_set_site2['connectionTime'] < end_of_hour) & (charging_set_site2['disconnectTime'] > start_of_hour)
    ]
    
    # calculate active minutes for each session
    active_minutes = 0
    for _, session in active_sessions.iterrows():
        session_start = max(session['connectionTime'], start_of_hour)
        session_end = min(session['disconnectTime'], end_of_hour)
        active_minutes += (session_end - session_start).total_seconds() / 60  # Convert to minutes
    
    # count number of cars
    connected_cars = len(active_sessions)
    # utilization as a percentage
    possible_minutes = total_spaces * 60  # 60 minutes per hour per space
    utilization = active_minutes / possible_minutes if possible_minutes > 0 else 0
    return utilization, connected_cars


# compute utilization column
hours_df2['utilization'] = hours_df2['hour'].apply(
    lambda h: calculate_utilization(h, charging_set_site2, total_spaces)[0]
)

# compute connectedCars column
hours_df2['connectedCars'] = hours_df2['hour'].apply(
    lambda h: calculate_utilization(h, charging_set_site2, total_spaces)[1]
)

# display results
hours_df2.head()

NameError: name 'hours_df2' is not defined

In [ ]:
hours_df.describe()

In [ ]:
hours_df2.describe()

---

## Merge the datasets on the timestamp columns

Merge Target Variable and Weather Dataset

In [ ]:
# transform column to the same format
hours_df["hour"] = hours_df["hour"].dt.tz_localize(None)
hours_df2["hour"] = hours_df2["hour"].dt.tz_localize(None)

# convert the "timestamp" column in the weather data to datetime
weather_set["timestamp"] = pd.to_datetime(weather_set["timestamp"])

# round the weather data timestamps to the nearest hour
weather_set["timestamp"] = weather_set["timestamp"].dt.floor("H")

# merge the datasets based on the "timestamp" column
merged_data_site1 = pd.merge(hours_df, weather_set, left_on="hour", right_on="timestamp", how="left")
merged_data_site2 = pd.merge(hours_df2, weather_set, left_on="hour", right_on="timestamp", how="left")
# drop the redundant "timestamp" column from the weather data after merging
merged_data_site1 = merged_data_site1.drop(columns=["timestamp"])
merged_data_site2 = merged_data_site2.drop(columns=["timestamp"])

In [ ]:
merged_data_site1.head()

In [ ]:
merged_data_site2.head()

???? drunter weil 2 mal Site 1

In [ ]:
# check dataset range
print(merged_data_site1["hour"].max(),merged_data_site1["hour"].max())

In [ ]:
# check NaN values 
print(merged_data_site1.isna().sum(),merged_data_site2.isna().sum())

In [ ]:
# drop NaN values
merged_data_site1 = merged_data_site1.dropna()
merged_data_site2 = merged_data_site2.dropna()

In [ ]:
# check if NaN values were dropped
print(merged_data_site1.isna().sum(),merged_data_site2.isna().sum())

In [ ]:
# check max date now
print(merged_data_site1["hour"].max(),merged_data_site2["hour"].max())

In [ ]:
# chec for duplicates
print(merged_data_site1.duplicated().sum(),merged_data_site2.duplicated().sum())

In [ ]:
merged_data_site1.describe()

In [ ]:
merged_data_site2.describe()

In [ ]:
start_date = "2020-08-02"
end_date = "2020-10-31"

filtered_data = merged_data_site1.loc[
    (merged_data_site1['hour'] >= pd.to_datetime(start_date)) &
    (merged_data_site1['hour'] <= pd.to_datetime(end_date))
]
filtered_data2 = merged_data_site2.loc[
    (merged_data_site2['hour'] >= pd.to_datetime(start_date)) &
    (merged_data_site2['hour'] <= pd.to_datetime(end_date))
]
print(len(filtered_data),len(filtered_data2))

In [ ]:
zero_connected_cars_count = filtered_data[filtered_data['connectedCars'] == 0].shape[0]
zero_connected_cars_count2 = filtered_data2[filtered_data2['connectedCars'] == 0].shape[0]
print(zero_connected_cars_count,zero_connected_cars_count2)

As we saw in the descriptive analysis (Section 2) there is a gap of data between august 2020 and Oktober 2020. This will most likely have a negativ impact on our model performance because as seen above those values are almost all 0. Therefore we are dropping them.

In [ ]:
merged_data_site1 = merged_data_site1.loc[
    ~((merged_data_site1['hour'] >= pd.to_datetime(start_date)) &
      (merged_data_site1['hour'] <= pd.to_datetime(end_date)))
]
merged_data_site2 = merged_data_site2.loc[
    ~((merged_data_site2['hour'] >= pd.to_datetime(start_date)) &
      (merged_data_site2['hour'] <= pd.to_datetime(end_date)))
]

In [ ]:
merged_data_site1.describe()

In [ ]:
merged_data_site2.describe()

---

### Feature Engineering for Modeling

Add the features to site 1 and site 2

In [ ]:
# create season column and weekend/weekday column and covvid column
merged_data_site1["hour"] = pd.to_datetime(merged_data_site1["hour"])
# add covid column
merged_data_site1["covid"] = merged_data_site1["hour"].apply(lambda x: 0 if x < pd.Timestamp("2020-03-01") else 1)

merged_data_site1["weekday"] = merged_data_site1["hour"].apply(lambda x: x.weekday())
 
# function to map months to seasons
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    elif month in [9, 10,11]:
        return "Fall"
    return np.nan

# apply the season mapping
merged_data_site1["Season"] = merged_data_site1["date_month"].apply(get_season)

# add weekday/weekend column 
merged_data_site1["Weekday/Weekend"] = merged_data_site1["hour"].apply(lambda x: 1 if x.weekday()>=5 else 0)
# mapping seasons to numeric values
season_mapping = {"Winter": 0, "Spring": 1, "Summer": 2, "Fall": 3}
merged_data_site1["Season"] = merged_data_site1["Season"].map(season_mapping)


# create df column for time of day
def assign_category(hour):
    if hour >= 23 or hour < 6:
        return 0  # Night
    elif 6 <= hour < 12:
        return 1  # Morning
    elif 12 <= hour < 18:
        return 2  # Afternoon
    else:
        return 3  # Evening

# apply the function to create a new column
merged_data_site1["time_of_day"] = merged_data_site1["hour"].dt.hour.apply(assign_category)

# create  precise hour of day column
merged_data_site1["hour_of_day"] = merged_data_site1["hour"].dt.hour

# resulting DataFrame
merged_data_site1.head()

In [ ]:
# create season column and weekend/weekday column and covvid column
merged_data_site2["hour"] = pd.to_datetime(merged_data_site2["hour"])
# add covid column
merged_data_site2["covid"] = merged_data_site2["hour"].apply(lambda x: 0 if x < pd.Timestamp("2020-03-01") else 1)

merged_data_site2["weekday"] = merged_data_site2["hour"].apply(lambda x: x.weekday())
 
# function to map months to seasons
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    elif month in [9, 10,11]:
        return "Fall"
    return np.nan

# apply the season mapping
merged_data_site2["Season"] = merged_data_site2["date_month"].apply(get_season)

# add weekday/weekend column 
merged_data_site2["Weekday/Weekend"] = merged_data_site2["hour"].apply(lambda x: 1 if x.weekday()>=5 else 0)
# mapping seasons to numeric values
season_mapping = {"Winter": 0, "Spring": 1, "Summer": 2, "Fall": 3}
merged_data_site2["Season"] = merged_data_site2["Season"].map(season_mapping)


# create df column for time of day
def assign_category(hour):
    if hour >= 23 or hour < 6:
        return 0  # Night
    elif 6 <= hour < 12:
        return 1  # Morning
    elif 12 <= hour < 18:
        return 2  # Afternoon
    else:
        return 3  # Evening

# apply the function to create a new column
merged_data_site2["time_of_day"] = merged_data_site2["hour"].dt.hour.apply(assign_category)

# create  precise hour of day column
merged_data_site2["hour_of_day"] = merged_data_site2["hour"].dt.hour
# resulting DataFrame
merged_data_site2.head()

### Feature Selection

Site 1:

In [4]:
# compute correlation matrix
correlation_matrix = merged_data_site1.corr()

# correlation with target feature
target_correlation = correlation_matrix['connectedCars'].sort_values(ascending=False)
print("Correlation with connectedCars:")
print(target_correlation)

NameError: name 'merged_data_site1' is not defined

In [ ]:
# compute correlation and display correlation for newly selected features
predictionDF_site1 = merged_data_site1[["connectedCars", "utilization","time_of_day","windspeed","temperature","felt_temperature","date_month","weekday","Weekday/Weekend","covid","hour_of_day"]]
correlation_matrix = predictionDF_site1.corr()
# plot heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

Site 2:

In [ ]:
# compute correlation matrix
correlation_matrix = merged_data_site2.corr()


# correlation with target feature
target_correlation = correlation_matrix['connectedCars'].sort_values(ascending=False)
print("Correlation with connectedCars:")
print(target_correlation)

In [ ]:
# compute correlation and display correlation for newly selected features
predictionDF_site2 = merged_data_site2[["connectedCars", "utilization","time_of_day","windspeed","temperature","felt_temperature","Season","date_month","weekday","Weekday/Weekend","covid","hour_of_day"]]
correlation_matrix = predictionDF_site2.corr()
# plot heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

### Prediction

#### ConnectedCars Prediction for Site 1 with Neural Network

In [ ]:
X = predictionDF_site1.copy()
X = predictionDF_site1.drop(["connectedCars","utilization","temperature","time_of_day"],axis = 1)
y = predictionDF_site1[["connectedCars"]]

In [ ]:
y.info()

In [ ]:
X.head()

In [ ]:
X.info()

In [ ]:
X.isna().sum()

In [ ]:
X = X.dropna()
y = y.dropna()

In [ ]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# standardize the feature values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# initialise model
model = Sequential(
    [Dense(160, activation="relu", input_shape=[X_train.shape[1]]),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
     Dense(32, activation="relu"),
      Dense(32, activation="relu"),
       Dense(32, activation="relu"),
       Dense(32, activation="relu"),
       Dense(32, activation="relu"),
     Dense(1)])

In [ ]:
# compile model
model.compile(optimizer="adam", loss = "mean_squared_error",
             metrics=["mae"])

In [ ]:
# train model
epochs = 65

history = model.fit(X_train, y_train.values,
                   epochs=epochs, validation_split=0.3)

In [ ]:
# make predictions
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)

In [ ]:
print(r2)

In [ ]:
# evaluating the model on test data
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test MAE: {mae}")

#### Connected Cars Prediction for Site 2

X = predictionDF_site2[["hour_of_day","covid","Weekday/Weekend","weekday","Season","temperature","windspeed"]]
y = predictionDF_site2[["connectedCars"]]

In [ ]:
X.head()

In [ ]:
y.info()

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the feature values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# initialise model
model = Sequential(
    [Dense(160, activation="relu", input_shape=[X_train.shape[1]]),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
     Dense(32, activation="relu"),
      Dense(32, activation="relu"),
       Dense(32, activation="relu"),
       Dense(32, activation="relu"),
       Dense(32, activation="relu"),
     Dense(1)])